In [ ]:
!pip install transformers
!pip install tqdm boto3 requests regex sentencepiece sacremoses
!pip install pytorch-transformers

In [4]:
import os
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import torch
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import  tqdm_notebook

In [5]:
hotel_df = pd.read_csv('data.csv')

In [ ]:
hotel_df.shape


In [8]:
hotel_df['REVIEW'] = hotel_df['data']

In [9]:
hotel_df['REVIEW'] = hotel_df.REVIEW.str.lower()

In [10]:
hotel_df['REVIEW'] = hotel_df['REVIEW'].map(lambda k: '[REVIEW] '+str(k))

In [11]:
changed_text=hotel_df.REVIEW.apply(lambda x:x+"\n"+"\n")

In [12]:
open('data_lm_hotel.txt', "w").write(''.join(changed_text))

160748

In [15]:
!python3 pregenerate_training_data.py --train_corpus data_lm_hotel.txt --bert_model bert-base-uncased --do_lower_case --output_dir training/ --epochs_to_generate 1 --max_seq_len 128

100% 231508/231508 [00:00<00:00, 26628683.13B/s]
Loading Dataset: 1998 lines [00:00, 4445.77 lines/s]
Epoch:   0% 0/1 [00:00<?, ?it/s]
Document: 100% 999/999 [00:00<00:00, 10020.09it/s]
Epoch: 100% 1/1 [00:00<00:00,  9.96it/s]


In [16]:
!python3 finetune_on_pregenerated.py --pregenerated_data training/ --bert_model bert-base-uncased --do_lower_case --train_batch_size 16  --output_dir finetuned_lm/ --epochs 1

2022-07-07 06:58:35,297: device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
2022-07-07 06:58:35,342: loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2022-07-07 06:58:35,423: https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpetrpbii1
100% 433/433 [00:00<00:00, 635233.87B/s]
2022-07-07 06:58:35,457: copying /tmp/tmpetrpbii1 to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
2022-07-07 06:58:35,457: creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250